In [1]:
import tifffile
import numpy as np
import random
from tkinter import *
from tkinter.ttk import *
import matplotlib.pyplot as plt

In [13]:
class DataAnnotate:
    def __init__(self, patch_num = 250, 
                 tif_path = r'/Users/bennysun/Desktop/IMG-Viewer/zstack_0.tif', 
                 label_path = r'/Users/bennysun/Desktop/IMG-Viewer/manual_data.tif' ):
        self.patch_num = patch_num
        
        #data type that looks at labels
        self.labels = tifffile.imread(label_path)
        
        #provides updated editings to the labels
        self.edited_labels = self.labels
        self.tif = tifffile.imread(tif_path)
        
        #defines whether all patches are finished or not 
        self.complete = False
        
        #will later add functionality for flexibile img sizes
        assert self.tif.shape[1] % patch_num == 0 and self.tif.shape[2] % patch_num == 0
        
    def create_data(self): 
        
        #creates dataframe for patches
        self.length_tiff = (int)(self.tif.shape[1]/self.patch_num)
        self.width_tiff = (int)(self.tif.shape[2]/self.patch_num)
        
        self.arr_length = self.length_tiff*self.width_tiff*self.tif.shape[0]
        
        self.arr = np.arange(0, self.arr_length, dtype =int)
        
    def get_patch(self, index, length, width):
        assert(index < self.tif.shape[0] and index >= 0)     
        
        #length calculation
        dim1_low = length * self.patch_num
        dim1_high = (length + 1)* self.patch_num
        
        #width calculation 
        dim2_low = width * self.patch_num
        dim2_high = (width + 1)* self.patch_num
        
        return self.tif[index][dim1_low:dim1_high,dim2_low:dim2_high]
    
    def get_random_patch(self):
        if(len(self.arr) == 0):
            print("All Patches are Complete!")
            self.complete = True
            return
        
        indx = random.randint(0, len(self.arr) - 1)
        
        val = self.arr[indx]
        self.arr = np.delete(self.arr, indx)
        
        index = int(val/(self.length_tiff * self.width_tiff))
        val = val % (self.length_tiff * self.width_tiff)
        length = int(val/(self.length_tiff))
        width = val % self.length_tiff
        
        return self.get_patch(index, length, width)

In [3]:
def image_transform(img, gamma):
    return img * 255 * (1/255)**(gamma)

In [4]:
test = DataAnnotate()
test.create_data()
x = test.get_random_patch()
edited_x = image_transform(x, 0.8)

In [5]:
#save and delete image
tifffile.imsave("temp.tif", x)
#os.remove("stupid.tif")

In [14]:
class ImageViewer(Tk):
    def __init__(self, patch_num = 250, 
                 tif_path = r'/Users/bennysun/Desktop/IMG-Viewer/zstack_0.tif', 
                 label_path = r'/Users/bennysun/Desktop/IMG-Viewer/manual_data.tif' ):
        super().__init__()
        
        self.data = DataAnnotate(patch_num, tif_path, label_path)
        self.data.create_data()
        
        #configure root window
        self.title("GUI Image Viewer")
        self.geometry("300x300")
        self.resizable(False, False)
        
        #self.y = self.data.get_random_patch()
        #tifffile.imsave("temp.tif", self.y)
        
        self.image1 = ImageTk.PhotoImage(Image.open("temp.tif"))
        self.label = Label(image = self.image1)
        self.label.grid(row = 1, column = 2, columnspan = 3)
        
        self.next_button = Button(text = "Next", command = self.next_click)
        self.next_button.grid(row = 2, column = 2)
    
    def next_click(self):
        self.label.grid_forget()
        if self.data.complete:
            self.label = Label(text = "All Photos Completed. Click Upload to Save!")
            self.label.grid(row = 1, column = 2, columnspan = 3)
            self.next_button = Button(text = "next", state = DISABLED)
            self.next_button.grid(row = 2, column = 2)
            return
        
        self.y = self.data.get_random_patch()
        tifffile.imsave("temp.tif", self.y)
        self.image1 = ImageTk.PhotoImage(Image.open("temp.tif"))
        self.label = Label(image = self.image1)
        self.label.grid(row = 1)
    
    #things to do: 1) Find Overlay + Brush Method 2) Create Save Feature 3) Add Erase 4) Update
        

In [15]:
stuff = ImageViewer()
stuff.mainloop()

In [ ]:
import os
from tkinter.colorchooser import askcolor
from tkinter import *
from PIL import ImageTk, Image

class FilenamePopup:
    def __init__(self, master):
        top = self.top = Toplevel(master)
        self.lbl = Label(top, text="Choose a file name:")
        self.lbl.pack()
        self.ent_filename = Entry(top)
        self.ent_filename.pack()
        self.btn_ok = Button(top, text='Ok', command=self.cleanup)
        self.btn_ok.pack()

    def cleanup(self):
        self.filename = self.ent_filename.get()
        self.top.destroy()


class Paint(object):
    DEFAULT_COLOR = 'black'

    def __init__(self):
        self.root = Tk()
        self.root.geometry("400x400")
        self.root.title("GUI Image Viewer")
        self.root.resizable(False, False)

        self.brush_button = Button(self.root, text='Brush',
                                   command=self.use_brush)
        self.brush_button.grid(row=0, column=0, sticky="ew")

        self.eraser_button = Button(self.root, text='Eraser',
                                    command=self.use_eraser)
        self.eraser_button.grid(row=0, column=1, sticky="ew")

        self.size_scale = Scale(self.root, from_=1, to=5,
                                orient='horizontal')
        self.size_scale.grid(row=2, column=1, sticky="ew")

        self.save_button = Button(self.root, text="Save",
                                  command=self.save_file)
        self.save_button.grid(row=2, column=0, sticky="ew")
          
        self.c = Canvas(self.root, width=250, height=250)
        self.image1 = ImageTk.PhotoImage(Image.open("stupid.tif"))
        self.c.create_image(125, 125, image = self.image1)
        self.c.grid(row=1)

        self.setup()
        self.root.mainloop()

    def setup(self):
        self.old_x, self.old_y = None, None
        self.color = self.DEFAULT_COLOR
        self.eraser_on = False
        self.active_button = None
        self.size_multiplier = 10

        self.activate_button(self.brush_button)
        self.c.bind('<B1-Motion>', self.paint)
        self.c.bind('<ButtonRelease-1>', self.reset)

        self.c.bind('<Button-1>', self.point)
        self.line_start = (None, None)

    def use_brush(self):
        self.activate_button(self.brush_button)

    def use_eraser(self):
        self.activate_button(self.eraser_button, eraser_mode=True)

    def activate_button(self, some_button, eraser_mode=False):
        if self.active_button:
            self.active_button.config(relief='raised')
        some_button.config(relief='sunken')
        self.active_button = some_button
        self.eraser_on = eraser_mode

    def paint(self, event):
        line_width = self.size_scale.get() * self.size_multiplier
        paint_color = 'white' if self.eraser_on else self.color
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x, self.old_y, event.x, event.y,
                               width=line_width, fill=paint_color,
                               capstyle='round', smooth=True, splinesteps=36)
        self.old_x = event.x
        self.old_y = event.y

    def point(self, event):
        btn = self.active_button["text"]
        if btn in ("Line", "Polygon"):
            self.size_multiplier = 1
            if any(self.line_start):
                self.line(event.x, event.y)
                self.line_start = ((None, None) if btn == 'Line'
                                   else (event.x, event.y))
            else:
                self.line_start = (event.x, event.y)

    def reset(self, event):
        self.old_x, self.old_y = None, None
    
    def color_default(self):
        self.color = self.DEFAULT_COLOR

    def save_file(self):
        self.popup = FilenamePopup(self.root)
        self.save_button["state"] = "disabled"
        self.root.wait_window(self.popup.top)

        filepng = self.popup.filename + '.png'

        if not os.path.exists(filepng) or \
                messagebox.askyesno("File already exists", "Overwrite?"):
            fileps = self.popup.filename + '.eps'

            self.c.postscript(file=fileps)
            img = Image.open(fileps)
            img = Image.open(fileps)
            img.save(filepng, 'png')
            os.remove(fileps)

            self.save_button["state"] = "normal"

            messagebox.showinfo("File Save", "File saved!")
        else:
            messagebox.showwarning("File Save", "File not saved!")

        self.save_button["state"] = "normal"

if __name__ == '__main__':
    Paint()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/bennysun/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-19-d7c997f11ee2>", line 122, in save_file
    img.save(filepng, 'png')
  File "/Users/bennysun/opt/anaconda3/lib/python3.8/site-packages/PIL/Image.py", line 2138, in save
    self._ensure_mutable()
  File "/Users/bennysun/opt/anaconda3/lib/python3.8/site-packages/PIL/Image.py", line 619, in _ensure_mutable
    self._copy()
  File "/Users/bennysun/opt/anaconda3/lib/python3.8/site-packages/PIL/Image.py", line 612, in _copy
    self.load()
  File "/Users/bennysun/opt/anaconda3/lib/python3.8/site-packages/PIL/EpsImagePlugin.py", line 332, in load
    self.im = Ghostscript(self.tile, self.size, self.fp, scale)
  File "/Users/bennysun/opt/anaconda3/lib/python3.8/site-packages/PIL/EpsImagePlugin.py", line 143, in Ghostscript
    subprocess.check_call(command, startupinfo=startupinfo)

In [6]:
label = r'/Users/bennysun/Desktop/IMG-Viewer/file.tif'
tifffile.imread(label)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/bennysun/Desktop/IMG-Viewer/file.tif'

In [10]:
from PyQt5.QtCore import QThread

In [11]:
 %gui qt5   

In [8]:
%gui qt5
import napari
from skimage import data

In [12]:
viewer = napari.Viewer()

In [20]:
def countP(n, k):
     
    # Table to store results of subproblems
    dp = [[0 for i in range(k + 1)]
             for j in range(n + 1)]
 
    # Base cases
    for i in range(n + 1):
        dp[i][0] = 0
 
    for i in range(k + 1):
        dp[0][k] = 0
 
    # Fill rest of the entries in
    # dp[][] in bottom up manner
    for i in range(1, n + 1):
        for j in range(1, k + 1):
            if (j == 1 or i == j):
                dp[i][j] = 1
            else:
                dp[i][j] = (j * dp[i - 1][j] +
                                dp[i - 1][j - 1])
                 
    return dp[n][k]
 
# Driver Code
if __name__ == '__main__':
    print(countP(11, 10))

55
